### Pomožne funkcije pri izdelavi pajka
 

In [2]:
import requests
from bs4 import BeautifulSoup



def robots_context(text):
    result_dict = {}
    lines = text.split('\n')
    sitemaps = []
    for line in lines:
        if line.startswith('Sitemap:'):
            if len(line.split())>1: sitemaps.append(line.split()[1])
        elif line.startswith('User-agent:') and line.split()[1] == '*':
            disallows = []
            allows = []
            for agent in lines[lines.index(line)+1:]:
                print(agent)
                if agent.strip() and  not agent.strip().startswith(('Disallow:', 'Allow:', '#')):
                    break
                if agent.startswith('Disallow:'):
                    if len(agent.split())>1: disallows.append(agent.split()[1])        
                if agent.startswith('Allow:'):
                    if len(agent.split())>1: allows.append(agent.split()[1]) 
            result_dict['User-agent'] = {'Disallow': disallows, 'Allow': allows}
        elif line.startswith('Crawl-delay:'):
            if len(line.split())>1: result_dict['Crawl-delay'] = line.split()[1]
        else:
            continue
    result_dict['Sitemap'] = sitemaps
    return result_dict

def main_page():
    #vstavimo neko stran ki ni prva.

    #kliče funkcijo, ki scrapa stran

    #pridobi osnovno stran in vstavi dobljene info v bazo

    #na izhodu dobimo:
    #id glavne -> mogoče samo to vrne, ker se lahko kliče bazo
    #vse strani iz sitemap
    pass



def url_sitemap(url):
    #za vstavlen url osnovne domene dobi vse povezave na strani
    response = requests.get(url + '/robots.txt')
    sitemap_list = robots_context(response.text)
    print(sitemap_list['Sitemap'])
    urls = []
    for sitemap in sitemap_list['Sitemap']:
        urls.extend(parse_sitemap(sitemap))
    return urls


def parse_sitemap(url_sitemap):
    #dobi urlje od sitmap xml datoteke
    urls = []

    #če je ni
    if url_sitemap is  None:
        return []

    # Fetch the sitemap XML file
    response = requests.get(url_sitemap)
    if response.status_code == 200:
        #preoblikujemo v leš
        soup = BeautifulSoup(response.content, 'xml')
        

    
        #išče <sitemap>, ki nakazuje na .xml datoteke
        sitemap = soup.find('sitemap')
        if sitemap is not None:
            
            
            for sitemap in soup.sitemapindex.find_all('sitemap'):
                # Extract the URLs from the sitemap
                loc = sitemap.loc.text
                
                #print(f'Location: {loc}')

                #če so xml datoteke
                if loc.endswith(".xml") or loc.endswith(".XML"):
                    urls.extend(parse_sitemap(loc))
                    
            #kliče sebe s parametrom, ki vsebuje ime xml datoteke z url-ji
        
        else:
            #datotka vsebuje url-je
            urlset = soup.find('urlset')
            if urlset is not None:
                url = soup.find('url')
                if url is not None:
                    urls.extend([element.text for element in soup.find_all('loc')])
       
  
    return urls


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://okusno.je/'  # Replace example.com with the actual domain


response = requests.get(url, headers=headers)
if response.status_code == 200:  # Check if the response is successful
    #print(len(url_sitemap(url)))
    resp = requests.get(url + '/robots.txt', headers=headers)

    print(robots_context(resp.text))
else:
    print('Failed to retrieve robots.txt')





Disallow: /iskanje
Disallow: /bin/
Disallow: /lbin/
Disallow: /adserver/
Disallow: *modal:*
Crawl-delay: 2
{'User-agent': {'Disallow': ['/iskanje', '/bin/', '/lbin/', '/adserver/', '*modal:*'], 'Allow': []}, 'Crawl-delay': '2', 'Sitemap': ['https://www.24ur.com/sitemaps/sites/10010']}


In [5]:
from urllib.parse import urlparse

websites = set()
web = "https://docs.python.org/3/library/"

# Odstrani duplikate glede na url:
#    HTTP vs HTTPS
#    www vs non-www
#    Trailing slash: example.com vs example.com/
def is_unique(website):
    parsed = urlparse(website)
    #odstrani zadnji / če obstaja
    path = parsed.path[:-1] if (parsed.path[-1] == "/") else parsed.path
    #docs.python.org/3/library
    url = parsed.hostname + path
    #primerja, če že obstaja obtoječa stran
    if url in websites:
        return False
    websites.add(url)
    print(url)
    return True

is_unique(web)

docs.python.org/3/library


True

In [10]:
import urllib 
from bs4 import BeautifulSoup
#pip install bs4
from langdetect import detect
#pip install langdetect
from nltk.corpus import stopwords
#pip install nltk
#python -m nltk.downloader stopwords or nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from iso639 import languages
#pip install iso-639




['C:\\Users\\sivar/nltk_data',
 'c:\\Users\\sivar\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data',
 'c:\\Users\\sivar\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data',
 'c:\\Users\\sivar\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data',
 'C:\\Users\\sivar\\AppData\\Roaming\\nltk_data',
 'C:\\nltk_data',
 'D:\\nltk_data',
 'E:\\nltk_data']

In [7]:
def recognizeLang(text):
    """
    Pomožna funkcija, ki pretvori pravilen zapis jezika
    """
    detLang = detect(text)
    
    if detLang == 'el':
        lang = "greek"
    elif detLang == 'zh-cn':
        lang = "chinese"
    elif detLang == 'zh-tw':
        lang = "chinese"
    else:
        lang = languages.get(alpha2=detLang).name.lower()
    return lang






def get_website_text(url_address):
    """
    Funkcija prebere besedilo strani in ga reducira na najmanjše možno obliko
    """

    with urllib.request.urlopen(url_address) as url:
        html = url.read()
        

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = ' '.join(chunk for chunk in chunks if chunk)
    sentence = text.lower()
   
    # poišče jezik vnešenega teksta iz prvih 100 besed
    reduced_sent = " ".join(sentence.split()[:100])
    lang = recognizeLang(reduced_sent)
    print(lang)

    # creates tokens, removes stopwords and punctation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    if not lang == "other":    
        filtered_words = [w for w in tokens if not w in stopwords.words(lang)]
    else:
        filtered_words =  [w for w in tokens]

    # odstrani podvojitve
    reduced_text = list(set(filtered_words))
    #print(reduced_text)

    return " ".join(reduced_text)



# dobljena s pomočjo chatGPT
def jaccard_similarity(url1, url2):
    """
    Calculate the Jaccard similarity coefficient between the text content of two websites.
    """
    # Get the text content of each website
    text1 = get_website_text(url1)
    text2 = get_website_text(url2)
    
    # Convert the text content to sets of words
    words1 = set(text1.split())
    words2 = set(text2.split())
    
    # Calculate the Jaccard similarity coefficient
    intersection = len(words1.intersection(words2))
    union = len(words1.union(words2))
    jaccard_similarity = intersection / union
    
    return jaccard_similarity



# This is an example of code which extracts all text from a web page and compares its content
url_address1 = "http://www.google.com"
url_address2 = "http://www.google.com/search"
jaccard_similarity(url_address1, url_address2)

slovenian


OSError: No such file or directory: 'C:\\Users\\sivar\\AppData\\Roaming\\nltk_data\\corpora\\stopwords\\slovenian'

In [9]:
stopwords.words("slovenian")

OSError: No such file or directory: 'C:\\Users\\sivar\\AppData\\Roaming\\nltk_data\\corpora\\stopwords\\slovenian'